# 1

Choose an 8-bit colored image and convert it to a grayscale image. Explain your
approach and show the input and output image.

In [1]:
from PIL import Image 
from tqdm import tqdm
import numpy as np

In [ ]:
# Opening the image and making a copy to not disturb the original

img = Image.open("8-bit-graphics-pixels-scene-with-village.jpg")
modded = img.copy()
modded = modded.convert("RGB")

width, height = img.size

So here we made every pixel in range of 0 and 1 for black and white. That is the reason for deviding into 255. We then multiplied with values that influence that specific area for example red with 0.299. Those values make up to 1 to get a gray scale of the image. If higher the image is brighter and if less than 1 the image becomes darker.

In [39]:
# Processing each pixel and converting to grayscale

for y in tqdm(range(height), desc="Processing image", ncols=80):
    for x in range(width):
        r, g, b = img.getpixel((x, y))
        
        r /= 255.0
        g /= 255.0
        b /= 255.0
        
        gray = (0.299 * r) + (0.587 * g) + (0.114 * b)
        
        gray_value = int(gray * 255)

        modded.putpixel((x, y), (gray_value, gray_value, gray_value))


Processing image: 100%|█████████████████████| 2688/2688 [00:36<00:00, 73.57it/s]


In [40]:
# Showing the gray image and original

print("Finished!")
modded.show()
img.show()

Finished!


# 2

Add white gaussian noise to the grayscale image. Show the effects of the noise when
the standard deviation is 1, 10 and 30 and 50.

In [ ]:
# function to add Gaussian noise to an image
def add_gNoise(imgArray, stdDev):
    noise = np.random.normal(0, stdDev, imgArray.shape)
    noisyImg = np.clip(imgArray + noise, 0, 255)
    return noisyImg.astype(np.uint8)

In [ ]:
# Converting the image to a numpy array for processing
imgArray = np.array(modded, dtype=float)
stdDevs = [1, 10, 30, 50]

So with helper function addgNoise helps repeat the same process to add the Gaussian white noise but using different standard deviations and seeing different effect of the noise into the gray scale image. The more deviation the more the image is affected with the noise effect.

In [ ]:
# Appluing different levels of standard deviation noise and showing the results

for std in stdDevs:
    noisy = add_gNoise(imgArray, std)
    noisyImg = Image.fromarray(noisy)
    noisyImg.show()

In [ ]:
# applying 50 standard deviation noise and showing the result

noisy = add_gNoise(imgArray, 50)
noisyImg = Image.fromarray(noisy)
noisyImg.show()

# 3

Add Salt and Pepper noise to 10% and 30% of the pixels in the grayscale image in 1.
Salt and Pepper noise is added to an image by adding random bright and random dark
pixels all over the image.



I reused the modded image and converted it into an array so that I can use it in the function.

In [ ]:
# making an array from the modded image again for future use
img_array = np.array(modded)

In [ ]:
# function to add Salt and Pepper noise to an image

def add_SP_Noise(imgArray, amnt):
    noisy = imgArray.copy()
    ttl_pxls = imgArray.size 
    num_noisy = int(ttl_pxls * amnt /2)

    coords = []

    for i in imgArray.shape:
        randomIs = np.random.randint(0, i - 1, num_noisy)
        coords.append(randomIs)

    noisy[coords[0], coords[1]] = 255

    coords = []
    for i in imgArray.shape:
        randomIs = np.random.randint(0, i - 1, num_noisy)
        coords.append(randomIs)

    noisy[coords[0], coords[1]] = 0        

    return noisy

So here we see how much salt and pepper we are going to apply to the picture and we see that 30% adds a good amount of noise into the picture.

In [47]:
# 10% noise
noisy10 = add_SP_Noise(img_array, 0.10)
noisy10_img = Image.fromarray(noisy10)
noisy10_img.show()

In [76]:
# 30% noise
noisy10 = add_SP_Noise(img_array, 0.30)
noisy10_img = Image.fromarray(noisy10)
noisy10_img.show()

# 4

Implement a Box Filter, Median Filter and Gaussian Filter to remove the white gaussian
noise with standard deviation 50 and the Salt and Pepper noise at 30%. Let the kernel
size be 3 x 3. Explain your choice of other parameters and any non-trivial steps in your
implementation (e.g., how did you handle the borders in the image).

To start we made this function to help fix the border issue and we used an edge method which where the empty locations that are outside of the border copy the values that are closest to that specific edge if that makes sense. 

In [ ]:
# function to pad an array with edge values because of the border issue

def pad_reflect(a, r=1):
    if a.ndim == 2:
        print(2)
        return np.pad(a, ((r, r), (r, r)), mode = "edge")
    elif a.ndim == 3:
        print(3)
        return np.pad(a, ((r, r), (r, r), (0, 0)), mode = "edge")

In [ ]:
# Setting up the array and applying guassian and salt and pepper noise

img_array = np.array(modded)
gauss = add_gNoise(img_array, stdDev = 50)
sp = add_SP_Noise(img_array, amnt= 0.30)

Work was separated here we started by doing the box filter first 3x3 and applied it to the gaussian noise picture using 50 as std deviation and 30% for the salt and pepper noise picture. I was working on RGB mode and gray scale mode so that is the reason there is the ndim = 2 and ndim = 3 conditional because different things are accounted when looking into the image shape.

In [ ]:
# mean box filter 3 x 3
def boxF3x3(img):
    k = np.ones((3, 3), dtype=float) / 9.0
    P = pad_reflect(img, r=1)

    if img.ndim == 2:
        H, W = img.shape
        out = np.empty((H, W), dtype = float)

        for y in range(H):
            for x in range(W):
                out[y, x] = np.sum(P[y:y+3, x:x+3] * k)
        return np.clip(out, 0, 255).astype(np.uint8)
    elif img.ndim == 3:
        H, W, C = img.shape 
        out = np.empty((H, W, C), dtype=float)

        for c in range(C):
            for y in range(H):
                for x in range(W):
                    out[y, x, c] = np.sum(P[y:y+3, x:x+3, c] * k)
        return np.clip(out, 0, 255).astype(np.uint8)
    else:
        print("Error")

In [62]:
# Applying the box filter to both noisy images 3x3

gaussb = boxF3x3(gauss)
spb = boxF3x3(sp)

3
3


For these results I'm not sure why but I didn't really see a difference in doing the 3x3 box filter on the gaussian noise and salt and pepper pictures. They look the same 

In [ ]:
# converting back to images and showing the results

gb_img = Image.fromarray(gaussb)
sp_img = Image.fromarray(spb)

gb_img.show()
sp_img.show()

Here is another helper function to do the median filter on both gaussian and salt and pepper noise images. Here I made the images into gray scale mode because when I would do the RGB mode which the picture is in gray scale so it doesn't really matter I switched because RGB mode is way slower because there is more computation to be made which doesn't really matter because the channels are pretty much the same for the R, G, and B.

In [149]:
# median filter 3 x 3

def medianF3x3(img, str):
    P = pad_reflect(img, r=1)

    if img.ndim == 2:
        H, W = img.shape
        out = np.empty((H, W), dtype = np.uint8)

        for y in tqdm(range(H), desc=str, ncols=80):
            for x in range(W):
                out[y, x] = np.median(P[y:y+3, x:x+3] )

        return np.clip(out, 0, 255).astype(np.uint8)
    
    elif img.ndim == 3:
        H, W, C = img.shape 
        out = np.empty((H, W, C), dtype=np.uint8)

        for c in tqdm(range(C), desc=str, ncols=100):
            for y in range(H):
                for x in range(W):
                    out[y, x, c] = np.median(P[y:y+3, x:x+3, c])
                    
        return np.clip(out, 0, 255).astype(np.uint8)
    else:
        print("Error")

In [150]:
# Converted the image into 2D so that it is faster to process since all channels are the same

grayg = gauss[..., 0]
gaussm = medianF3x3(grayg, "Processing Median for Gauss: ")

grays = sp[..., 0]
spm = medianF3x3(grays, "Processing Median for SP: ")

2


Processing Median for Gauss: 100%|██████████| 2688/2688 [03:02<00:00, 14.76it/s]


2


Processing Median for SP: 100%|█████████████| 2688/2688 [03:02<00:00, 14.73it/s]


The end result looked weird I'm not completely sure if this is what I was supposed to get because I thought it was going to remove the noise for the salt and pepper noise image and it didn't really. I guess I might have done something wrong in my logic.

In [151]:
# converting back to images and showing the results

gm_img = Image.fromarray(gaussm)
spm_img = Image.fromarray(spm)

gm_img.show()
spm_img.show()

So in these helper functions I only did an implementation to do the Gaussian filter in gray scale mode rather than RGB.

In [ ]:
# Gaussian Kernel 3 x 3

def gK3x3(sigma = 0.85):
    ax = np.array([-1, 0, 1], dtype=float)
    xx, yy = np.meshgrid(ax, ax)
    k = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    k /= k.sum()
    return k

In [ ]:
# Gaussian Filter 3 x 3

def gF3x3(img, sigma = 0.85):
    k = gK3x3(sigma)
    P = pad_reflect(img, r=1)
    H, W = img.shape 
    out = np.empty((H, W), dtype = float)
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(P[y:y+3, x:x+3] * k)
    out = np.clip(out, 0, 255).astype(np.uint8)
    return out

In [80]:
# Applying Gaussian filter to both noisy images 3x3

guass_g = gF3x3(grayg, sigma = 0.85)
sp_g = gF3x3(grays, sigma = 0.85)

ggf_img = Image.fromarray(guass_g)
sgf_img = Image.fromarray(sp_g)

ggf_img.show()
sgf_img.show()

2
2


# 5

Vary the kernel size to 5 x 5 and 10 x 10 and show its effect on the output image.

Here I just did the same thing as previous steps but made the filters modular to accept different sizes of Kernel size. This helps to reuse and redo these things over and over again with different situtaions

In [131]:
# Mean Box Filter k x k for reusability on 5x5 and 10x10

def boxF(img, kSize, str="processing Box Filter"):
    k = np.ones((kSize, kSize), dtype=float) / (kSize * kSize)
    r = kSize // 2
    P = pad_reflect(img, r=r)

    if img.ndim == 2:
        H, W = img.shape
        out = np.empty((H, W), dtype = float)

        for y in tqdm(range(H), desc=str, ncols=80):
            for x in range(W):
                out[y, x] = np.sum(P[y:y+kSize, x:x+kSize] * k)
        return np.clip(out, 0, 255).astype(np.uint8)
    elif img.ndim == 3:
        H, W, C = img.shape 
        out = np.empty((H, W, C), dtype=float)
        with tqdm(total=H * W * C, desc=str, ncols=100) as pbar:
            for c in range(C):
                for y in range(H):
                    for x in range(W):
                        out[y, x, c] = np.sum(P[y:y+kSize, x:x+kSize, c] * k)
                        pbar.update(1)
        return np.clip(out, 0, 255).astype(np.uint8)
    else:
        print("Error")

In [132]:
# Median Filter k x k for reusability on 5x5 and 10x10

def medianF(img, kSize, str = "Processing median"):
    P = pad_reflect(img, r=1)

    if img.ndim == 2:
        H, W = img.shape
        out = np.empty((H, W), dtype = np.uint8)

        for y in tqdm(range(H), desc=str, ncols=80):
            for x in range(W):
                out[y, x] = np.median(P[y:y+kSize, x:x+kSize] )

        return np.clip(out, 0, 255).astype(np.uint8)
    
    elif img.ndim == 3:
        H, W, C = img.shape 
        out = np.empty((H, W, C), dtype=np.uint8)
        with tqdm(total=H * W * C, desc=str, ncols=100) as pbar:
            for c in range( C):
                for y in range(H):
                    for x in range(W):
                        out[y, x, c] = np.median(P[y:y+kSize, x:x+kSize, c])
                        pbar.update(1)
                    
        return np.clip(out, 0, 255).astype(np.uint8)
    else:
        print("Error")

In [ ]:
# Gaussian Kernel k x k for reusability on 5x5 and 10x10

def gK(kSize, sigma=1):
    assert kSize % 2 == 1 and kSize >= 3
    r = kSize // 2
    ax = np.arange(-r, r+1, dtype=float)
    xx, yy = np.meshgrid(ax, ax)
    k = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    k /= k.sum()
    return k, r


In [ ]:
# Gaussian Filter k x k for reusability on 5x5 and 10x10

def gFilter(img, kSize, sigma=1, str = "Processing Guass: "):
    if kSize % 2 == 0:
        kSize += 1 # make it odd if even because of center pixel 10x10 -> 11x11
    k, r = gK(kSize, sigma)
    r = int(r)
    P = pad_reflect(img, r=r)

    if img.ndim == 2:
        H, W = img.shape
        out = np.empty((H, W), dtype=float)
        for y in tqdm(range(H), desc=str, ncols=80):
            for x in range(W):
                out[y, x] = np.sum(P[y:y+kSize, x:x+kSize] * k)
        return np.clip(out, 0, 255).astype(np.uint8)
    elif img.ndim == 3:
        H, W, C = img.shape 
        out = np.empty((H, W, C), dtype=float)
        with tqdm(total=H * W * C, desc=str, ncols=100) as pbar:
            for c in range(C):
                for y in range(H):
                    for x in range(W):
                        out[y, x, c] = np.sum(P[y:y+kSize, x:x+kSize, c] * k)
                        pbar.update(1)
        return np.clip(out, 0, 255).astype(np.uint8)

In [140]:
# applying the three filters with kernel sizes of 5 and 10 to the salt and pepper noisy image

spR = []
sp2d = sp[..., 0]

for kS in [5, 10]:
    spb = boxF(sp2d, kS, "Processing Box: ")
    spM = medianF(sp2d, kS, "Processing median: ")
    spg = gFilter(sp2d, kS, 1, "Processing Guass: ")
    spR.append((spb, spM, spg))


2


Processing Box: 100%|███████████████████████| 2688/2688 [02:09<00:00, 20.71it/s]


2


Processing median: 100%|████████████████████| 2688/2688 [05:01<00:00,  8.90it/s]


2


...

Processing Box: 100%|███████████████████████| 2688/2688 [02:07<00:00, 21.01it/s]


2


Processing median: 100%|████████████████████| 2688/2688 [05:37<00:00,  7.96it/s]


2


Processing Guass: 100%|█████████████████████| 2688/2688 [02:04<00:00, 21.60it/s]


So I was reviewing the output the images and the Box filter and the Gaussian kind of look similar. When I look from afar the Gaussian filter image I can see the picture kind of clear. The picture that removed most of the noise was the median filter for 5x5 and 10x10. I guess the bigger the size of the kernel the more the salt and pepper noise it removes because of the median thing we are doing in that box. I now understand why in the 3x3 the noise wasn't really taken off because the kernel size was too small to actually do an impact of removing the salt and pepper noise. I'm not sure about the other 2 filters how I can reflect and talk about them though. 

In [ ]:
# showing all the results for 5x5 and 10x10

for b, m, g in spR:
    b_img = Image.fromarray(b)
    m_img = Image.fromarray(m)
    g_img = Image.fromarray(g)

    b_img.show(title="Box Filter")
    m_img.show(title="Median Filter")
    g_img.show(title="Guassian Filter")

In [144]:
# applying the three filters with kernel sizes of 5 and 10 to the Gaussian noisy image

gR = []
gauss2d = gauss[..., 0]
for kS in [5, 10]:
    gb = boxF(gauss2d, kS, "Processing Box: ")
    gM = medianF(gauss2d, kS, "Processing median: ")
    gg = gFilter(gauss2d, kS, 1, "Processing Gauss: ")
    gR.append((gb, gM, gg))

2


Processing Box: 100%|███████████████████████| 2688/2688 [01:03<00:00, 42.42it/s]


2


Processing median: 100%|████████████████████| 2688/2688 [03:02<00:00, 14.76it/s]


2


...

Processing Box: 100%|███████████████████████| 2688/2688 [01:08<00:00, 39.22it/s]


2


Processing median: 100%|████████████████████| 2688/2688 [03:30<00:00, 12.78it/s]


2


Processing Gauss: 100%|█████████████████████| 2688/2688 [01:08<00:00, 39.16it/s]


To be honest I'm not sure if i did something wrong here too. I feel like all the picture here in the Gaussian noise using all three filters look the same. It looks like using different kernel sizes don't really matter. I mean if I reflect from these images and the one we used in step 4 they do look different because in step 4 I remember seeing like small specs of some like rgb noise but in these new images I don't see that noise anymore rather than black and white noise specifically black noise in the pictures.

In [154]:
# showing all the results for 5x5 and 10x10

for b, m, g in gR:
    b_img = Image.fromarray(b)
    m_img = Image.fromarray(m)
    g_img = Image.fromarray(g)

    b_img.show(title="Box Filter")
    m_img.show(title="Median Filter")
    g_img.show(title="Guassian Filter")